<a href="https://colab.research.google.com/github/kartikgandhi/Soft-Voting-Ensemble-Results-Edited-Dataset/blob/main/Ensemble_model_Freeplane.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [218]:
from numpy import mean
from numpy import std

In [219]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [220]:
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier

In [221]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [222]:
DATA = pd.read_csv("/content/gdrive/MyDrive/Edited Datasets/Edited-freeplane 1.7.9 - freeplane 1.7.9.csv")


In [223]:
DATA['change'] = DATA['change'].map({'YES': 1, 'NO': 0})

In [224]:
DATA.shape

(1278, 8)

In [225]:
DATA.head()

,CBO,NOC,RFC,LOC,DIT,LCOM,WMC,change
0,6,0,12,155,1,70,32,0
1,9,0,5,118,1,50,22,0
2,3,0,2,20,2,0,4,0
3,2,0,1,7,2,0,1,0
4,0,0,24,128,2,0,27,0


In [226]:
X = DATA.drop('change', axis=1)
y = DATA['change']

In [227]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

In [228]:
# get a voting ensemble of models
def get_voting():
	# define the base models
	models = list()
	models.append(('lin', SVC(probability=True, kernel='linear')))
	models.append(('rbf', SVC(probability=True, kernel='rbf')))
	models.append(('poly', SVC(probability=True, kernel='poly')))

	# define the voting ensemble
	ensemble = VotingClassifier(estimators=models, voting='soft')
	return ensemble

In [229]:
# get a list of models to evaluate
def get_models():
	models = dict()
	models['hard_voting'] = get_voting()
	return models

In [230]:
# evaluate a give model
def evaluate_model(model, X, y):
  model=model.fit(X_train,y_train) 
  y_pred = model.predict(X_test)
  return y_pred

In [231]:
# get the models to evaluate
models = get_models()

In [232]:
results, names = list(), list()
for name, model in models.items():
    y_pred = evaluate_model(model, X, y)
    results.append(y_pred)
    names.append(name)
    print('>%s %.3f (%.3f)' % (name, mean(y_pred), std(y_pred)))

>hard_voting 0.008 (0.088)


In [233]:
print(y_pred)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [234]:
from sklearn.metrics import classification_report, confusion_matrix
cm=confusion_matrix(y_test,y_pred)
print(cm)
print(classification_report(y_test, y_pred))

[[238   1]
 [ 16   1]]
              precision    recall  f1-score   support

           0       0.94      1.00      0.97       239
           1       0.50      0.06      0.11        17

    accuracy                           0.93       256
   macro avg       0.72      0.53      0.54       256
weighted avg       0.91      0.93      0.91       256



In [235]:
# predict probabilities
from sklearn.metrics import roc_auc_score
pred_prob = model.predict_proba(X_test)
auc_score = roc_auc_score(y_test, pred_prob[:,1])
print("Area Under Curve=")
print(auc_score)

Area Under Curve=
0.7784888013782918


In [236]:
from sklearn.metrics import matthews_corrcoef
mcc=matthews_corrcoef(y_test,y_pred)
print("Matthews correlation coefficient=")
print(mcc)

Matthews correlation coefficient=
0.15452462814004875


In [237]:
from sklearn.metrics import balanced_accuracy_score
bac=balanced_accuracy_score(y_test, y_pred)
print("Balanced Accuracy Score=")
print(bac)

Balanced Accuracy Score=
0.5273197144966774


In [238]:
from imblearn.metrics import geometric_mean_score
gmean=geometric_mean_score(y_test, y_pred)
print("Geometric Mean Score=")
print(gmean)

Geometric Mean Score=
0.24202769646827732
